In [1]:
#import

import requests
import json
import pandas as pd
import hashlib
import random

In [51]:
#블록 정보 조회
headers = {"Content-Type" : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)


{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1715134282.1701427,
   'transactions': []},
  {'index': 2,
   'nonce': 269826,
   'previous_hash': '7f6337e96bf9c82a7b9dc025001760e582039153dae39bf4b61c9f2a9a9c01e2',
   'timestamp': 1715134382.0758789,
   'transactions': [{'amount': 3,
     'recipient': 'test_to',
     'sender': 'test_from',
     'timestamp': 1715134369.889754},
    {'amount': 3,
     'recipient': 'test_to',
     'sender': 'test_from',
     'timestamp': 1715134378.882044},
    {'amount': 0.1,
     'recipient': 'node_8000',
     'sender': 'master',
     'timestamp': 1715134382.0758789}]}],
 'length': 2}

In [58]:
#transaction 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender" : "test_from",
        "recipient" : "test_to",
    "amount" : 3,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data = json.dumps(data)).content

b'{"message":"Transaction will be added to Block {2}"}\n'

In [50]:
#채굴하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)
print(res.text)

<Response [200]>
{"index":2,"message":"new block found","nonce":269826,"previous_hash":"7f6337e96bf9c82a7b9dc025001760e582039153dae39bf4b61c9f2a9a9c01e2","transactions":[{"amount":3,"recipient":"test_to","sender":"test_from","timestamp":1715134369.889754},{"amount":3,"recipient":"test_to","sender":"test_from","timestamp":1715134378.882044},{"amount":0.1,"recipient":"node_8000","sender":"master","timestamp":1715134382.0758789}]}



In [52]:
#transaction2 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender" : "test_from",
    "recipient" : "test_to2",
    "amount" : 30,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

b'{"message":"Transaction will be added to Block {3}"}\n'

In [53]:
#transaction3 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender" : "test_from",
    "recipient" : "test_to3",
    "amount" : 300,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

b'{"message":"Transaction will be added to Block {3}"}\n'

In [54]:
#채굴하기
headers = {"Content-Type" : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

<Response [200]>


In [55]:
#노드의 블록 정보 확인 -4
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)

In [57]:
res.text

'{"chain":[{"index":1,"nonce":100,"previous_hash":1,"timestamp":1715134282.1701427,"transactions":[]},{"index":2,"nonce":269826,"previous_hash":"7f6337e96bf9c82a7b9dc025001760e582039153dae39bf4b61c9f2a9a9c01e2","timestamp":1715134382.0758789,"transactions":[{"amount":3,"recipient":"test_to","sender":"test_from","timestamp":1715134369.889754},{"amount":3,"recipient":"test_to","sender":"test_from","timestamp":1715134378.882044},{"amount":0.1,"recipient":"node_8000","sender":"master","timestamp":1715134382.0758789}]},{"index":3,"nonce":667285,"previous_hash":"e25f6d7040744e520df0a8789082f3b3b086ab5a28c735ac921ef90b099816d6","timestamp":1715134413.7049396,"transactions":[{"amount":30,"recipient":"test_to2","sender":"test_from","timestamp":1715134407.6591313},{"amount":300,"recipient":"test_to3","sender":"test_from","timestamp":1715134410.5000675},{"amount":0.1,"recipient":"node_8000","sender":"master","timestamp":1715134413.7049396}]}],"length":3}\n'

In [43]:
#거래 내역 확인
status_json = json.loads(res.text)
status_json['chain']
tx_amount_l = []
tx_sender_l = []
tx_reciv_l  = []
tx_time_l   = []


for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])


df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l
df_tx['sender'] = tx_sender_l
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l
df_tx

,timestamp,sender,recipient,amount
0,1.715134e+09,test_from,test_to,3.0
1,1.715134e+09,master,node_8000,0.1
2,1.715134e+09,test_from,test_to2,30.0
3,1.715134e+09,test_from,test_to3,300.0
4,1.715134e+09,master,node_8000,0.1


In [44]:
#지갑별 잔액 계산
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended,columns = ['user', 'sended_amount']
df_received = pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_sended,columns = ['user', 'received_amount']
df_received



,recipient,amount
0,node_8000,0.2
1,test_to,3.0
2,test_to2,30.0
3,test_to3,300.0


In [45]:
df_status = pd.merge(df_received, df_sended, on = 'user', how= 'outer').fillna(0)
df_status['balance'] = df_status['received_amount'] - df_status['sended_amount']
df_status

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed